In [1]:
pip install tensorflow scikit-learn pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

data = {
    "sequence": [
        "DOWN,RIGHT,PUNCH",
        "RIGHT,DOWN,RIGHT,KICK",
        "LEFT,LEFT,PUNCH",
        "DOWN,KICK",
        "UP,PUNCH",
        "RIGHT,RIGHT,KICK",
        "DOWN,DOWN,RIGHT,PUNCH",
        "LEFT,DOWN,RIGHT,PUNCH"
    ],
    "move": [
        "Hadouken",
        "Shoryuken",
        "Dash Punch",
        "Low Kick",
        "Jump Punch",
        "Double Kick",
        "Super Hadouken",
        "Combo Strike"
    ]
}

df = pd.DataFrame(data)


In [3]:
# Tokenizer ve Label Encoding

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Joystick hareketlerini tokenize et
tokenizer = Tokenizer(filters='', lower=False, split=',')
tokenizer.fit_on_texts(df['sequence'])

X_seq = tokenizer.texts_to_sequences(df['sequence'])
X_pad = pad_sequences(X_seq, padding='post')  # sekansları eşitle

# Etiketleri sayısallaştır
le = LabelEncoder()
y_encoded = le.fit_transform(df['move'])

# Bilgiler
vocab_size = len(tokenizer.word_index) + 1
num_classes = len(le.classes_)


In [4]:
 # LSTM Modeli Oluştur ve Eğit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=16, input_length=X_pad.shape[1]),
    LSTM(32),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_pad, y_encoded, epochs=100, verbose=1)


Epoch 1/100


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 2.0767
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2500 - loss: 2.0758
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3750 - loss: 2.0749
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5000 - loss: 2.0739
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6250 - loss: 2.0730
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 2.0720
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6250 - loss: 2.0710
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5000 - loss: 2.0699
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5000 - loss: 2.0689
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5000 - loss: 2.0677
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5000 - loss: 2.0665
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5000 - loss: 2.0653
Epo

In [6]:
# Tahmin Fonksiyonu

import numpy as np

def predict_move(sequence_text):
    seq = tokenizer.texts_to_sequences([sequence_text])
    pad = pad_sequences(seq, maxlen=X_pad.shape[1], padding='post')
    pred = model.predict(pad)
    label = le.inverse_transform([np.argmax(pred)])
    return label[0]

# Örnek:
predict_move("DOWN,RIGHT,PUNCH")  # Hadouken


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step


'Hadouken'

In [7]:
model.save("joystick_move_model.keras")


In [8]:
import pickle

# Tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# LabelEncoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)
